In [7]:
import pandas as pd
import numpy as np
import pickle
from src import helpers
import math
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Google Documentation

In [8]:
len([0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8])

17

In [9]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

n_nodes = 17
n_vehicles = 16

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data['demands'] = [1 for i in range(n_nodes)]
    data['vehicle_capacities'] = [math.ceil(n_nodes/n_vehicles) for i in range(n_vehicles)]
    data["num_vehicles"] = n_vehicles
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 15520
Route for vehicle 0:
 0 Load(1) ->  2 Load(2) ->  0 Load(2)
Distance of the route: 1552m
Load of the route: 2

Route for vehicle 1:
 0 Load(1) ->  15 Load(2) ->  0 Load(2)
Distance of the route: 1552m
Load of the route: 2

Route for vehicle 2:
 0 Load(1) ->  3 Load(2) ->  0 Load(2)
Distance of the route: 1392m
Load of the route: 2

Route for vehicle 3:
 0 Load(1) ->  16 Load(2) ->  0 Load(2)
Distance of the route: 1324m
Load of the route: 2

Route for vehicle 4:
 0 Load(1) ->  4 Load(2) ->  0 Load(2)
Distance of the route: 1164m
Load of the route: 2

Route for vehicle 5:
 0 Load(1) ->  1 Load(2) ->  0 Load(2)
Distance of the route: 1096m
Load of the route: 2

Route for vehicle 6:
 0 Load(1) ->  10 Load(2) ->  0 Load(2)
Distance of the route: 1072m
Load of the route: 2

Route for vehicle 7:
 0 Load(1) ->  6 Load(2) ->  0 Load(2)
Distance of the route: 1004m
Load of the route: 2

Route for vehicle 8:
 0 Load(1) ->  11 Load(2) ->  0 Load(2)
Distance of the route: 1004m
Lo

Learning point, if you didn't set the capacity constraints, it may not utilise all the vehicles. Reason is it's cheaper to send a vehicles close by to a location than it is to dispatch it from the depot and back again. 

# My Locations

In [10]:
fp = 'data/pairwise_dist_matrix.pkl'
pairwise_dist_matrix = helpers.read_pickle(fp)
pairwise_dist_matrix_df = pd.DataFrame(pairwise_dist_matrix)
pairwise_dist_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.0,20995.0,25439.0,25463.0,17623.0,22841.0,13945.0,16524.0,3667.0,19713.0,...,22924.0,12774.0,10472.0,20062.0,26858.0,23457.0,18116.0,13367.0,19671.0,25263.0
1,20995.0,0.0,21213.0,38108.0,20880.0,26646.0,21613.0,27861.0,18737.0,9412.0,...,19685.0,23286.0,23312.0,10212.0,19062.0,16770.0,15326.0,9151.0,9370.0,6320.0
2,25439.0,21213.0,0.0,21948.0,3850.0,7309.0,8709.0,13177.0,20996.0,11701.0,...,811.0,11270.0,12992.0,11710.0,5119.0,4217.0,5578.0,17684.0,11351.0,18983.0
3,25463.0,38108.0,21948.0,0.0,18434.0,12917.0,17297.0,12932.0,25066.0,30591.0,...,22095.0,16909.0,20765.0,30940.0,25110.0,26365.0,21541.0,33535.0,28617.0,37735.0
4,17623.0,20880.0,3850.0,18434.0,0.0,5093.0,5597.0,9829.0,18894.0,12084.0,...,4251.0,7922.0,7345.0,12141.0,9199.0,9114.0,4230.0,15582.0,11305.0,23018.0
5,22841.0,26646.0,7309.0,12917.0,5093.0,0.0,7890.0,7812.0,22653.0,17560.0,...,7907.0,8618.0,10340.0,17569.0,10493.0,10938.0,9524.0,19877.0,17209.0,24842.0
6,13945.0,21613.0,8709.0,17297.0,5597.0,7890.0,0.0,6662.0,14022.0,13625.0,...,8841.0,2620.0,3762.0,13928.0,12852.0,11469.0,6128.0,15963.0,13092.0,20505.0
7,16524.0,27861.0,13177.0,12932.0,9829.0,7812.0,6662.0,0.0,17217.0,19688.0,...,13443.0,4585.0,7115.0,19992.0,17799.0,17714.0,12191.0,22027.0,19155.0,31618.0
8,3667.0,18737.0,20996.0,25066.0,18894.0,22653.0,14022.0,17217.0,0.0,17337.0,...,20548.0,13141.0,10840.0,17686.0,24482.0,21081.0,15740.0,10991.0,17294.0,22887.0
9,19713.0,9412.0,11701.0,30591.0,12084.0,17560.0,13625.0,19688.0,17337.0,0.0,...,9240.0,15258.0,15284.0,877.0,9468.0,7176.0,7766.0,9806.0,607.0,7469.0


In [11]:
# My locations
my_locations = pd.read_csv("data/locations_w_node_id.csv")
my_locations.head()

,id,location_name,location_type,lat,lon,node_id
0,school_1,Admiralty Primary School,school,1.44294,103.80035,8655379195
1,school_2,Jurong West Primary School,school,1.33931,103.69903,7437327501
2,school_3,River Valley Primary School,school,1.29430,103.83593,11938018645
3,school_4,Casuarina Primary School,school,1.37281,103.95723,8302208139
4,school_5,St. Joseph’s Institution Junior,school,1.31751,103.84632,10979316519


In [12]:
subset_list = [i for i in range(31)]
pairwise_dist_matrix_df = pairwise_dist_matrix_df.astype(int)
subset = pairwise_dist_matrix_df.iloc[subset_list, subset_list]
subset.values.tolist()

[[0,
  20995,
  25439,
  25463,
  17623,
  22841,
  13945,
  16524,
  3667,
  19713,
  2712,
  21725,
  17616,
  20074,
  27251,
  25619,
  24470,
  16631,
  20540,
  15706,
  956,
  22924,
  12774,
  10472,
  20062,
  26858,
  23457,
  18116,
  13367,
  19671,
  25263],
 [20995,
  0,
  21213,
  38108,
  20880,
  26646,
  21613,
  27861,
  18737,
  9412,
  23538,
  19141,
  32032,
  9502,
  19455,
  21989,
  36247,
  24297,
  1054,
  11084,
  21132,
  19685,
  23286,
  23312,
  10212,
  19062,
  16770,
  15326,
  9151,
  9370,
  6320],
 [25439,
  21213,
  0,
  21948,
  3850,
  7309,
  8709,
  13177,
  20996,
  11701,
  23572,
  1831,
  18193,
  12200,
  5297,
  3621,
  20087,
  9101,
  20294,
  10060,
  24756,
  811,
  11270,
  12992,
  11710,
  5119,
  4217,
  5578,
  17684,
  11351,
  18983],
 [25463,
  38108,
  21948,
  0,
  18434,
  12917,
  17297,
  12932,
  25066,
  30591,
  23332,
  20969,
  9439,
  30952,
  25289,
  18335,
  1421,
  13414,
  35631,
  26024,
  24516,
  22095,
  

In [13]:
"""Simple Vehicles Routing Problem (VRP).

This is a sample using the routing library python wrapper to solve a VRP
problem.
A description of the problem can be found here:
http://en.wikipedia.org/wiki/Vehicle_routing_problem.

Distances are in meters.
"""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = (np.array(subset.values.tolist())/1000).astype(int).tolist()
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        100000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")


if __name__ == "__main__":
    main()

Objective: 5505
Route for vehicle 0:
 0 ->  6 ->  17 ->  5 ->  15 ->  11 ->  27 ->  23 -> 0
Distance of the route: 49m

Route for vehicle 1:
 0 ->  1 ->  18 ->  30 ->  28 ->  8 -> 0
Distance of the route: 50m

Route for vehicle 2:
 0 ->  19 ->  13 ->  24 ->  9 ->  29 ->  26 ->  25 ->  14 ->  21 ->  2 ->  4 -> 0
Distance of the route: 53m

Route for vehicle 3:
 0 ->  10 ->  22 ->  7 ->  16 ->  3 ->  12 ->  20 -> 0
Distance of the route: 53m

Maximum of the route distances: 53m


In [14]:
my_locations

,id,location_name,location_type,lat,lon,node_id
0,school_1,Admiralty Primary School,school,1.44294,103.80035,8655379195
1,school_2,Jurong West Primary School,school,1.33931,103.69903,7437327501
2,school_3,River Valley Primary School,school,1.29430,103.83593,11938018645
3,school_4,Casuarina Primary School,school,1.37281,103.95723,8302208139
4,school_5,St. Joseph’s Institution Junior,school,1.31751,103.84632,10979316519
5,school_6,Geylang Methodist School Primary,school,1.31845,103.88414,8157277817
6,school_7,Whitley Secondary School,school,1.35546,103.84266,5251084191
7,school_8,Xinmin Secondary School,school,1.37256,103.88293,380363672
8,school_9,Fuchun Secondary School,school,1.43072,103.77730,10844240865
9,school_10,Clementi Town Secondary School,school,1.31566,103.76191,2391178715


## Try my locations with capacity constraints

In [15]:
(np.array(subset.values.tolist())/1000).astype(int).tolist()

[[0,
  20,
  25,
  25,
  17,
  22,
  13,
  16,
  3,
  19,
  2,
  21,
  17,
  20,
  27,
  25,
  24,
  16,
  20,
  15,
  0,
  22,
  12,
  10,
  20,
  26,
  23,
  18,
  13,
  19,
  25],
 [20,
  0,
  21,
  38,
  20,
  26,
  21,
  27,
  18,
  9,
  23,
  19,
  32,
  9,
  19,
  21,
  36,
  24,
  1,
  11,
  21,
  19,
  23,
  23,
  10,
  19,
  16,
  15,
  9,
  9,
  6],
 [25,
  21,
  0,
  21,
  3,
  7,
  8,
  13,
  20,
  11,
  23,
  1,
  18,
  12,
  5,
  3,
  20,
  9,
  20,
  10,
  24,
  0,
  11,
  12,
  11,
  5,
  4,
  5,
  17,
  11,
  18],
 [25,
  38,
  21,
  0,
  18,
  12,
  17,
  12,
  25,
  30,
  23,
  20,
  9,
  30,
  25,
  18,
  1,
  13,
  35,
  26,
  24,
  22,
  16,
  20,
  30,
  25,
  26,
  21,
  33,
  28,
  37],
 [17,
  20,
  3,
  18,
  0,
  5,
  5,
  9,
  18,
  12,
  16,
  3,
  14,
  16,
  9,
  4,
  16,
  5,
  17,
  8,
  17,
  4,
  7,
  7,
  12,
  9,
  9,
  4,
  15,
  11,
  23],
 [22,
  26,
  7,
  12,
  5,
  0,
  7,
  7,
  22,
  17,
  20,
  7,
  12,
  18,
  10,
  5,
  11,
  4,
  23,
 

In [16]:
[math.ceil(n_nodes/n_vehicles) for i in range(n_vehicles)]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [17]:
subset_list = [i for i in range(31)]
pairwise_dist_matrix_df = pairwise_dist_matrix_df.astype(int)
subset = pairwise_dist_matrix_df.iloc[subset_list, subset_list]
subset.values.tolist()

[[0,
  20995,
  25439,
  25463,
  17623,
  22841,
  13945,
  16524,
  3667,
  19713,
  2712,
  21725,
  17616,
  20074,
  27251,
  25619,
  24470,
  16631,
  20540,
  15706,
  956,
  22924,
  12774,
  10472,
  20062,
  26858,
  23457,
  18116,
  13367,
  19671,
  25263],
 [20995,
  0,
  21213,
  38108,
  20880,
  26646,
  21613,
  27861,
  18737,
  9412,
  23538,
  19141,
  32032,
  9502,
  19455,
  21989,
  36247,
  24297,
  1054,
  11084,
  21132,
  19685,
  23286,
  23312,
  10212,
  19062,
  16770,
  15326,
  9151,
  9370,
  6320],
 [25439,
  21213,
  0,
  21948,
  3850,
  7309,
  8709,
  13177,
  20996,
  11701,
  23572,
  1831,
  18193,
  12200,
  5297,
  3621,
  20087,
  9101,
  20294,
  10060,
  24756,
  811,
  11270,
  12992,
  11710,
  5119,
  4217,
  5578,
  17684,
  11351,
  18983],
 [25463,
  38108,
  21948,
  0,
  18434,
  12917,
  17297,
  12932,
  25066,
  30591,
  23332,
  20969,
  9439,
  30952,
  25289,
  18335,
  1421,
  13414,
  35631,
  26024,
  24516,
  22095,
  

In [18]:
(np.array(subset.values.tolist())/1000).astype(int).tolist()

[[0,
  20,
  25,
  25,
  17,
  22,
  13,
  16,
  3,
  19,
  2,
  21,
  17,
  20,
  27,
  25,
  24,
  16,
  20,
  15,
  0,
  22,
  12,
  10,
  20,
  26,
  23,
  18,
  13,
  19,
  25],
 [20,
  0,
  21,
  38,
  20,
  26,
  21,
  27,
  18,
  9,
  23,
  19,
  32,
  9,
  19,
  21,
  36,
  24,
  1,
  11,
  21,
  19,
  23,
  23,
  10,
  19,
  16,
  15,
  9,
  9,
  6],
 [25,
  21,
  0,
  21,
  3,
  7,
  8,
  13,
  20,
  11,
  23,
  1,
  18,
  12,
  5,
  3,
  20,
  9,
  20,
  10,
  24,
  0,
  11,
  12,
  11,
  5,
  4,
  5,
  17,
  11,
  18],
 [25,
  38,
  21,
  0,
  18,
  12,
  17,
  12,
  25,
  30,
  23,
  20,
  9,
  30,
  25,
  18,
  1,
  13,
  35,
  26,
  24,
  22,
  16,
  20,
  30,
  25,
  26,
  21,
  33,
  28,
  37],
 [17,
  20,
  3,
  18,
  0,
  5,
  5,
  9,
  18,
  12,
  16,
  3,
  14,
  16,
  9,
  4,
  16,
  5,
  17,
  8,
  17,
  4,
  7,
  7,
  12,
  9,
  9,
  4,
  15,
  11,
  23],
 [22,
  26,
  7,
  12,
  5,
  0,
  7,
  7,
  22,
  17,
  20,
  7,
  12,
  18,
  10,
  5,
  11,
  4,
  23,
 

In [ ]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

n_nodes = 31
n_vehicles = 4

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = subset.values.tolist()
    data['demands'] = [0] + [1 for i in range(n_nodes-1)]
    print([math.ceil(n_nodes/n_vehicles) for i in range(n_vehicles)])
    data['vehicle_capacities'] = [math.ceil(n_nodes/n_vehicles) for i in range(n_vehicles)]
    data["num_vehicles"] = n_vehicles
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    def get_routes(solution, routing, manager):
        """Get vehicle routes from a solution and store them in an array."""
        # Get vehicle routes and store them in a two dimensional array whose
        # i,j entry is the jth location visited by vehicle i along its route.
        routes = []
        for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
                index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        routes = get_routes(solution, routing, manager)
        return routes
    else:
        print('No solution found.')


if __name__ == "__main__":
    routes = main()
    


[8, 8, 8, 8]
Objective: 229218
Route for vehicle 0:
 0 Load(0) ->  29 Load(1) ->  9 Load(2) ->  24 Load(3) ->  13 Load(4) ->  30 Load(5) ->  18 Load(6) ->  1 Load(7) ->  28 Load(8) ->  0 Load(8)
Distance of the route: 58384m
Load of the route: 8

Route for vehicle 1:
 0 Load(0) ->  20 Load(1) ->  12 Load(2) ->  3 Load(3) ->  16 Load(4) ->  5 Load(5) ->  15 Load(6) ->  11 Load(7) ->  27 Load(8) ->  0 Load(8)
Distance of the route: 71803m
Load of the route: 8

Route for vehicle 2:
 0 Load(0) ->  4 Load(1) ->  2 Load(2) ->  21 Load(3) ->  14 Load(4) ->  25 Load(5) ->  26 Load(6) ->  19 Load(7) ->  8 Load(8) ->  0 Load(8)
Distance of the route: 58472m
Load of the route: 8

Route for vehicle 3:
 0 Load(0) ->  23 Load(1) ->  6 Load(2) ->  17 Load(3) ->  7 Load(4) ->  22 Load(5) ->  10 Load(6) ->  0 Load(6)
Distance of the route: 40559m
Load of the route: 6

Total distance of all routes: 229218m
Total load of all routes: 30


In [20]:
routes

[[0, 29, 9, 24, 13, 30, 18, 1, 28, 0],
 [0, 20, 12, 3, 16, 5, 15, 11, 27, 0],
 [0, 4, 2, 21, 14, 25, 26, 19, 8, 0],
 [0, 23, 6, 17, 7, 22, 10, 0]]

In [27]:
print('-> '.join(my_locations.loc[routes[0]]['location_name'].tolist()))

Admiralty Primary School-> Clementi 448 Market & Food Centre-> Clementi Town Secondary School-> Ayer Rajah Food Centre-> NEWest-> Jurong Port-> Jurong Point-> Jurong West Primary School-> Keat Hong Food Centre & Market-> Admiralty Primary School
